In [3]:
# !pip install mord

In [4]:
# !pip install dmba

In [5]:
from pathlib import Path

import numpy as np
import pandas as pd

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#from mord import LogisticIT
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm

from dmba import classificationSummary, gainsChart, liftChart, regressionSummary

%matplotlib inline
import matplotlib.pylab as plt

In [6]:

hospital_df = pd.read_csv('hospital_readmissions.csv')

# Display the first 10 records of bank_df data frame.
print(hospital_df.head(10))

print('')
print('---------------------------------')
print('')

# Identify data types of all variables.
print(hospital_df.dtypes)

print('')
print('---------------------------------')
print('')

# Identify shape of the data frame.
print(hospital_df.shape)

       age  time_in_hospital  n_lab_procedures  n_procedures  n_medications  \
0  [70-80)                 8                72             1             18   
1  [70-80)                 3                34             2             13   
2  [50-60)                 5                45             0             18   
3  [70-80)                 2                36             0             12   
4  [60-70)                 1                42             0              7   
5  [40-50)                 2                51             0             10   
6  [50-60)                 4                44             2             21   
7  [60-70)                 1                19             6             16   
8  [80-90)                 4                67             3             13   
9  [70-80)                 8                37             1             18   

   n_outpatient  n_inpatient  n_emergency       medical_specialty  \
0             2            0            0                 Mis

In [7]:

#dropping records because dataset is too big

hospital_df = hospital_df.drop(hospital_df.sample(frac=0.75, random_state=42).index)

print(hospital_df.shape)

(6250, 17)


In [8]:

# Print columns names.
print('Original column names:')
print(hospital_df.columns)

Original column names:
Index(['age', 'time_in_hospital', 'n_lab_procedures', 'n_procedures',
       'n_medications', 'n_outpatient', 'n_inpatient', 'n_emergency',
       'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'glucose_test',
       'A1Ctest', 'change', 'diabetes_med', 'readmitted'],
      dtype='object')


In [9]:

############# Q.1(c) ###########

# The following variables are of 'object' type and do not have  the 'category' definition:
#age, medical_specialty, diag_1, diag_2, diag_3, glucose_test, A1Ctest, change, diabetes_med, readmitted,

# Change all object type variable to 'category'.
hospital_df[['age', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'glucose_test',
       'A1Ctest', 'change', 'diabetes_med', 'readmitted']] = hospital_df[['age', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'glucose_test',
       'A1Ctest', 'change', 'diabetes_med', 'readmitted']].astype('category')

# Display category classes.
print(' ')
print('Category levels and changed variable type:')
{col: hospital_df[col].cat.categories.tolist() for col in hospital_df.columns if isinstance(hospital_df[col].dtype, pd.CategoricalDtype)}


 
Category levels and changed variable type:


{'age': ['[40-50)', '[50-60)', '[60-70)', '[70-80)', '[80-90)', '[90-100)'],
 'medical_specialty': ['Cardiology',
  'Emergency/Trauma',
  'Family/GeneralPractice',
  'InternalMedicine',
  'Missing',
  'Other',
  'Surgery'],
 'diag_1': ['Circulatory',
  'Diabetes',
  'Digestive',
  'Injury',
  'Missing',
  'Musculoskeletal',
  'Other',
  'Respiratory'],
 'diag_2': ['Circulatory',
  'Diabetes',
  'Digestive',
  'Injury',
  'Missing',
  'Musculoskeletal',
  'Other',
  'Respiratory'],
 'diag_3': ['Circulatory',
  'Diabetes',
  'Digestive',
  'Injury',
  'Missing',
  'Musculoskeletal',
  'Other',
  'Respiratory'],
 'glucose_test': ['high', 'no', 'normal'],
 'A1Ctest': ['high', 'no', 'normal'],
 'change': ['no', 'yes'],
 'diabetes_med': ['no', 'yes'],
 'readmitted': ['no', 'yes']}

In [10]:

# Object type variables are now converted to 'category' type.
print(hospital_df.dtypes)

age                  category
time_in_hospital        int64
n_lab_procedures        int64
n_procedures            int64
n_medications           int64
n_outpatient            int64
n_inpatient             int64
n_emergency             int64
medical_specialty    category
diag_1               category
diag_2               category
diag_3               category
glucose_test         category
A1Ctest              category
change               category
diabetes_med         category
readmitted           category
dtype: object


In [11]:

# Convert all category variable to dummy integer variables

hospital_df = pd.get_dummies(hospital_df, drop_first=True, dtype='int64')
hospital_df = hospital_df.astype('int64')

print(hospital_df.dtypes)

time_in_hospital                            int64
n_lab_procedures                            int64
n_procedures                                int64
n_medications                               int64
n_outpatient                                int64
n_inpatient                                 int64
n_emergency                                 int64
age_[50-60)                                 int64
age_[60-70)                                 int64
age_[70-80)                                 int64
age_[80-90)                                 int64
age_[90-100)                                int64
medical_specialty_Emergency/Trauma          int64
medical_specialty_Family/GeneralPractice    int64
medical_specialty_InternalMedicine          int64
medical_specialty_Missing                   int64
medical_specialty_Other                     int64
medical_specialty_Surgery                   int64
diag_1_Diabetes                             int64
diag_1_Digestive                            int64


In [12]:

# Display first 5 records.
print(hospital_df.head(5))

    time_in_hospital  n_lab_procedures  n_procedures  n_medications  \
9                  8                37             1             18   
11                 4                69             0              6   
13                 3                60             0             18   
16                 3                52             0             10   
24                 4                37             2             15   

    n_outpatient  n_inpatient  n_emergency  age_[50-60)  age_[60-70)  \
9              0            0            0            0            0   
11             0            0            0            1            0   
13             0            2            0            0            0   
16             0            0            0            0            0   
24             0            0            0            1            0   

    age_[70-80)  ...  diag_3_Musculoskeletal  diag_3_Other  \
9             1  ...                       0             1   
11            0 

In [13]:

# Apply all predictors and outcome
# for multi predictor neural network model.
predictors = hospital_df.drop(columns=['readmitted_yes'])
outcome = hospital_df['readmitted_yes']

# Create predictor X and outcome y variables.
X = predictors
y = outcome

# Partition data into training (70% or 0.7) and validation(30% or 0.3)
# of the accident_upd data frame.
train_X, valid_X, train_y, valid_y = train_test_split(X, y,
                            test_size=0.3, random_state=1)

# Display the first 10 records of Hospital Readmission training partition's predictors.
print('Predictor records for Training Partition')
print(train_X.head(5))

print('')
print('---------------------------------------------------')
print('')

print('Outcome records for Training Partition')
print(train_y.head(5))

Predictor records for Training Partition
       time_in_hospital  n_lab_procedures  n_procedures  n_medications  \
14755                 2                48             0             10   
6283                  1                34             0              4   
18047                 5                19             0             17   
20713                 1                31             5             15   
808                   5                75             5             17   

       n_outpatient  n_inpatient  n_emergency  age_[50-60)  age_[60-70)  \
14755             0            0            0            0            0   
6283              0            1            0            1            0   
18047             0            0            0            1            0   
20713             0            0            0            1            0   
808               0            1            0            0            0   

       age_[70-80)  ...  diag_3_Missing  diag_3_Musculoskeletal

In [14]:

# Scale input data (predictors) for training  and validation
# partitions using StandardScaler().
sc_X = StandardScaler()
train_X_sc = sc_X.fit_transform(train_X)
valid_X_sc = sc_X.transform(valid_X)

# Develop a data frame to display scaled predictors for
# training partition. Round scaled values to 3 decimals.
# Add coloumn titles to data frame.
train_X_sc_df = pd.DataFrame(np.round(train_X_sc, 3), columns=train_X.columns)

# Display scaled predictors for training partition.
print()
print('Scaled Predictors for Training Partition:')
print(train_X_sc_df.head(5))


Scaled Predictors for Training Partition:
   time_in_hospital  n_lab_procedures  n_procedures  n_medications  \
0            -0.819             0.241        -0.781         -0.772   
1            -1.158            -0.470        -0.781         -1.526   
2             0.198            -1.232        -0.781          0.108   
3            -1.158            -0.622         2.183         -0.144   
4             0.198             1.613         2.183          0.108   

   n_outpatient  n_inpatient  n_emergency  age_[50-60)  age_[60-70)  \
0        -0.333       -0.522       -0.192       -0.465        -0.56   
1        -0.333        0.301       -0.192        2.149        -0.56   
2        -0.333       -0.522       -0.192        2.149        -0.56   
3        -0.333       -0.522       -0.192        2.149        -0.56   
4        -0.333        0.301       -0.192       -0.465        -0.56   

   age_[70-80)  ...  diag_3_Missing  diag_3_Musculoskeletal  diag_3_Other  \
0       -0.611  ...           -0

In [15]:

# Use MLPRegressor() function to train neural network model.
# Apply:
# (a) default input layer with the number of nodes equal
#     to number of predictor variables (45);
# (b) single hidden layer with 40 nodes;
# (c) default output layer with one outcome variable (MVALUE);
# (d) optimization function solver = 'lbfgs',
#     which is applied for small data sets for better
#     performance and fast convergence. For large data sets,
#     apply default solver = 'adam' optimization function;
# (e) model is fit with scaled predictors and regular outcome
#     in training partition.
hospital_reg = MLPRegressor(hidden_layer_sizes=(20),
                solver='lbfgs', max_iter=10000, random_state=1)
hospital_reg.fit(train_X_sc, train_y)

# Display network structure with the final values of
# intercepts (Theta) and weights (W).
print('Final Intercepts for Hospital Readmission Neural Network Model')
print(hospital_reg.intercepts_)

print()
print('Network Weights for Hospital Readmission Neural Network Model')
print(hospital_reg.coefs_)

Final Intercepts for Hospital Readmission Neural Network Model
[array([-0.13382692, -0.32705677, -0.32341174,  0.32812926,  0.01950466,
        0.01413828,  0.29372219, -0.39160841, -0.23475831, -0.06540316,
        0.57575916, -0.58383911, -0.09282722,  0.08503826,  0.2956193 ,
        0.13718262, -0.0961369 ,  0.50938553, -0.11376415,  0.08993338]), array([-0.24266256])]

Network Weights for Hospital Readmission Neural Network Model
[array([[ 1.94016029e-01, -4.46890264e-02, -2.68760235e-01,
        -1.63798950e-01, -5.55263297e-01,  8.71055694e-02,
        -2.70130139e-02, -5.20525018e-02,  1.03467347e-01,
        -1.11274938e-01,  1.74896651e-01,  4.31969465e-01,
         9.19083814e-02, -6.02009461e-02, -3.07351936e-01,
        -1.36132203e-02, -5.88076878e-01,  1.38134826e-01,
        -4.37431583e-01, -2.53376680e-01],
       [-1.85019704e-01,  3.50987629e-02, -6.41478261e-01,
         1.85257091e-01,  1.00912429e-01,  7.89835399e-01,
        -1.44232533e-01, -1.52201999e-01, -4.

In [16]:

# Make 'readmitted_yes' predictions for validation set using Hospital Readmission
# neural network model.

# Use hospital_reg model to predict 'readmitted_yes' outcome
# for validation set.
readmitted_yes_pred = np.round(hospital_reg.predict(valid_X_sc), decimals=2)

# Create data frame to display prediction results for
# validation set.
readmitted_yes_pred_result = pd.DataFrame({'Actual': valid_y,
                'Prediction': readmitted_yes_pred, 'Residual': valid_y - readmitted_yes_pred})

print('Predictions for Readmissions for Validation Partition')
print(readmitted_yes_pred_result.head(10))

Predictions for Readmissions for Validation Partition
       Actual  Prediction  Residual
4758        0        0.37     -0.37
11168       1        0.77      0.23
18767       0       -0.13      0.13
22415       1        0.25      0.75
20022       1        0.41      0.59
18030       1        0.25      0.75
10106       1        0.10      0.90
22179       0        0.57     -0.57
22146       1        0.18      0.82
15688       1        0.56      0.44


In [17]:

# Neural network model accuracy measures for training and
# validation partitions. 

# Identify and display neural network model accuracy measures 
# for training partition.
print('Accuracy Measures for Training Partition for Neural Network')
regressionSummary(train_y, hospital_reg.predict(train_X_sc))

# Identify and display neural network accuracy measures 
# for validation partition.
print()
print('Accuracy Measures for Validation Partition for Neural Network')
regressionSummary(valid_y, hospital_reg.predict(valid_X_sc))

Accuracy Measures for Training Partition for Neural Network

Regression statistics

               Mean Error (ME) : 0.0000
Root Mean Squared Error (RMSE) : 0.3932
     Mean Absolute Error (MAE) : 0.3342

Accuracy Measures for Validation Partition for Neural Network

Regression statistics

               Mean Error (ME) : 0.0065
Root Mean Squared Error (RMSE) : 0.5675
     Mean Absolute Error (MAE) : 0.4867


In [18]:
# Use MLPCclassifier() function for nerual network model.
# Apply a default single hidden layer with 3 nodes, 'logistic'
# activation function, and solver = 'lbfgs', which is
# applied for small data sets for better performance
# and fast convergence. For large data sets, apply default
# solver = 'adam'.
tiny_clf = MLPClassifier(hidden_layer_sizes=(3), activation='logistic',
                    solver='lbfgs', random_state=1)
tiny_clf.fit(X, y)

# Make predictions using neural network model for
# Tiny data set.
tiny_clf.predict(X)

# Display network structure with the final values of
# intercepts (Theta) and weights (W).
print('Final Intercepts for Tiny Neural Network Model')
print(tiny_clf.intercepts_)

print()
print('Network Weights for Tiny Neural Network Model')
print(tiny_clf.coefs_)


Final Intercepts for Tiny Neural Network Model
[array([-0.95884637,  0.13956732, -0.38813904]), array([-1.34122902])]

Network Weights for Tiny Neural Network Model
[array([[ 0.01381458,  0.1160514 ,  0.01782113],
       [-0.02165236, -0.32213813, -0.08768355],
       [ 0.07701118, -0.09896288,  0.38227847],
       [ 0.09195134, -0.46453036,  0.20200026],
       [ 0.34109831,  0.28905485, -0.1223363 ],
       [ 0.89056212,  0.03646975, -0.57864229],
       [ 0.61003086, -0.07503437, -0.25995732],
       [ 0.19168632, -0.07080697, -0.15209852],
       [ 0.3773057 ,  0.17699781, -0.26159924],
       [ 0.71889465, -0.16317188,  0.47953982],
       [ 0.75140678, -0.02098381,  0.05710284],
       [ 0.31222673,  0.09450926, -0.18695758],
       [ 0.23464948,  0.16543782, -0.43102881],
       [ 0.41313097,  0.19794786, -0.0868647 ],
       [-0.13133227,  0.10582053, -0.03241139],
       [ 0.22354664,  0.17216285, -0.11373619],
       [-0.22643547, -0.16606328, -0.23593553],
       [-0.2017433

c:\Users\STSC\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [19]:

# Make accident severity classification for validation set 
# using Accidents neural network model. 

# Use accident_clf model to classify accident severity
# for validation set.
hospital_pred = tiny_clf.predict(valid_X)

# Predict accident severity probabilities p(0), p(1),
# and p(2) for validation set.
hospital_pred_prob = np.round(tiny_clf.predict_proba(valid_X), 
                          decimals=4)

# Create data frame to display classification results for
# validation set. 
hospital_pred_result = pd.DataFrame({'Actual': valid_y, 
                'p(0)': [p[0] for p in hospital_pred_prob],
                'p(1)': [p[1] for p in hospital_pred_prob],
                'Classification': hospital_pred})

print('Classification for readmission Data for Validation Partition')
print(hospital_pred_result.head(10))

Classification for readmission Data for Validation Partition
       Actual    p(0)    p(1)  Classification
4758        0  0.6108  0.3892               0
11168       1  0.2747  0.7253               1
18767       0  0.5516  0.4484               0
22415       1  0.5578  0.4422               0
20022       1  0.4883  0.5117               1
18030       1  0.5504  0.4496               0
10106       1  0.7692  0.2308               0
22179       0  0.7033  0.2967               0
22146       1  0.4046  0.5954               1
15688       1  0.5980  0.4020               0


In [20]:
# Confusion matrices for Accidents neural network model. 

# Identify and display confusion matrix for training partition. 
print('Training Partition for Neural Network Model')
classificationSummary(train_y, tiny_clf.predict(train_X))

# Identify and display confusion matrix for validation partition. 
print()
print('Validation Partition for Neural Network Model')
classificationSummary(valid_y, tiny_clf.predict(valid_X))

Training Partition for Neural Network Model
Confusion Matrix (Accuracy 0.6176)

       Prediction
Actual    0    1
     0 1645  649
     1 1024 1057

Validation Partition for Neural Network Model
Confusion Matrix (Accuracy 0.6192)

       Prediction
Actual   0   1
     0 702 273
     1 441 459


## Grid search for Hospital Readmission

In [21]:

# Identify grid search parameters. 
param_grid = {
    'hidden_layer_sizes': list(range(2, 20)), 
}

# Utilize GridSearchCV() to identify the best number 
# of nodes in the hidden layer. 
gridSearch = GridSearchCV(MLPClassifier(solver='lbfgs', max_iter=10000, random_state=1), 
                          param_grid, cv=5, n_jobs=-1, scoring='accuracy') 
gridSearch.fit(train_X, train_y)

# Display the best score and best parament value.
print(f'Best score:{gridSearch.best_score_:.4f}')
print('Best parameter: ', gridSearch.best_params_)

Best score:0.6048
Best parameter:  {'hidden_layer_sizes': 6}


In [22]:
# Use MLPCclassifier() function to train the improved neural network model
# based on grid search results. 

# Apply: 
# (a) default input layer with the number of nodes equal 
#     to number of predictor variables (7); 
# (b) single hidden layer with 10 nodes based on grid search; 
# (c) default output layer with the number nodes equal
#     to number of classes in outcome variable (3);
# (d) 'logistic' activation function;
# (e) solver = 'lbfgs', which is applied for small data 
#     sets for better performance and fast convergence. 
#     For large data sets, apply default solver = 'adam'. 
hospital_clf_imp = MLPClassifier(hidden_layer_sizes=(5), max_iter=1000,
                activation='logistic', solver='lbfgs', random_state=1)
hospital_clf_imp.fit(train_X, train_y)

# Display network structure with the final values of 
# intercepts (Theta) and weights (W).
print('Final Intercepts for Hospital Readmission Neural Network Model')
print(hospital_clf_imp.intercepts_)

print()
print('Network Weights for Hospital Readmission Neural Network Model')
print(hospital_clf_imp.coefs_)

Final Intercepts for Hospital Readmission Neural Network Model
[array([ 0.03359235, -0.15473965,  0.14271732, -0.47779837,  0.69634787]), array([2.42826624])]

Network Weights for Hospital Readmission Neural Network Model
[array([[-2.08535513e-01, -8.51103476e-01, -2.92863012e-01,
         4.31505340e-02,  6.85255747e-01],
       [-6.77237919e-02,  1.01343678e-03, -6.78993519e-01,
         5.81246354e-03, -3.45632124e-01],
       [ 4.51073790e-01,  8.40488578e-01, -2.16814490e-01,
        -7.88096854e-02, -9.43351788e-02],
       [ 1.52843030e-01, -5.22511822e-01, -8.53178202e-01,
        -1.99596755e-02, -7.57546814e-01],
       [ 5.17688813e-01,  7.86946913e-01, -4.03420800e-02,
        -3.87687153e-01,  1.78537858e-01],
       [-7.47275207e-01,  2.78011156e-01, -1.37011284e-01,
        -3.30684362e-01, -2.10587206e+00],
       [ 1.61177338e-01,  1.10618971e-01,  1.99036841e-01,
        -3.04101319e-01, -1.38866754e-01],
       [ 2.76060181e-01, -3.46061892e+00,  1.64547862e-01,
    

c:\Users\STSC\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [23]:
# Confusion matrices for improved neural network model for Accidents
# using grid search results. 

# Identify and display confusion matrix for training partition. 
print('Training Partition for Neural Network Model')
classificationSummary(train_y, hospital_clf_imp.predict(train_X))

# Identify and display confusion matrix for validation partition. 
print()
print('Validation Partition for Neural Network Model')
classificationSummary(valid_y, hospital_clf_imp.predict(valid_X))

Training Partition for Neural Network Model
Confusion Matrix (Accuracy 0.6402)

       Prediction
Actual    0    1
     0 1720  574
     1 1000 1081

Validation Partition for Neural Network Model
Confusion Matrix (Accuracy 0.6048)

       Prediction
Actual   0   1
     0 685 290
     1 451 449


In [24]:
# For each hidden_layer_size, display grid search results including
# mean and standard deviation of the score. 
display=['param_hidden_layer_sizes', 'mean_test_score', 'std_test_score']
print(pd.DataFrame(gridSearch.cv_results_)[display])

    param_hidden_layer_sizes  mean_test_score  std_test_score
0                          2         0.544457        0.037555
1                          3         0.584457        0.033096
2                          4         0.524343        0.000457
3                          5         0.600229        0.006680
4                          6         0.604800        0.008348
5                          7         0.596343        0.007832
6                          8         0.593829        0.008841
7                          9         0.591771        0.006872
8                         10         0.572343        0.007278
9                         11         0.590857        0.014954
10                        12         0.579429        0.014510
11                        13         0.590171        0.011047
12                        14         0.584000        0.010890
13                        15         0.586971        0.013611
14                        16         0.588800        0.013855
15      

In [25]:
# Develop a plot that demostrates mean_test_score and std_test_score.
pd.DataFrame(gridSearch.cv_results_)[display].plot(x='param_hidden_layer_sizes', 
                    y='mean_test_score', yerr='std_test_score', ylim=(0.5, 0.9))

<Axes: xlabel='param_hidden_layer_sizes'>